# TAREA 4

### PANDAS: Zona de habitabilidad de exoplanetas 

Base de datos de exoplanetas encontrados hasta el momento.



In [ ]:
import pandas as pd
import numpy as np

In [ ]:
exo = pd.read_csv("exoplanets.csv")

In [ ]:
#Constantes a Utilizar

sigma = 5.670374419*(10**(-8)) * (696350000)**2  #Constante de Stefan–Boltzmann en radios solares
sun_lum = 3.83*10**26   #luminosidad del Sol

Ts = 5780 
ai = 2.7619e-5
bi = 3.8095e-9
ao = 1.3786e-4
bo = 1.4286e-9
ris = 0.72
ros = 1.77

In [ ]:
exo.head()

In [ ]:
exo_zone = exo[["NAME","TEFF", "MASS", "A", "DENSITY", "R", "STAR", "MSTAR", "RSTAR", "BINARY"]]
exo_zone.head()

Cada columna coresponde a:

- NAME
- TEFF: temperatura efectiva de la estrella.
- MASS: Masa del planeta.
- A: semieje mayor.
- DENSITY: densidad.
- R: radio del planeta.
- STAR: Nombre de la estrella.
- MSTAR: Masa de la estrella.
- RSTAR: Radio de la estrella.
- BINARY: indica si el sistema es binario o no.

In [ ]:
#Observar los datos de la columna que dice si el sistema es binario o no
exo_zone["BINARY"].value_counts()

In [ ]:
#Descarte los sistemas binarios i.e. BINARY==0.
exo_zone = exo_zone[exo_zone["BINARY"] != 0.0]
exo_zone["BINARY"].value_counts()

In [ ]:
#MASSE: La masa de los planetas de masas de Jupiter a unidades de masa de la Tierra -> 1MJ = 317.8MT
exo_zone["MASSE"] = exo_zone[["MASS"]]*(317.8) 

In [ ]:
#RE: El radio de los planetas de radios de Jupiter a radios terrestres -> 1RJ = 11.209RT
exo_zone["RE"] = exo_zone[["R"]]*(11.209)

In [ ]:
# LUM: Luminosidad de una estrella (L = 4πR^2σT^4), R es el radio de la estrella, σ la constante de Stefan-Boltzmann
#Luminosidad está Watts, por lo tanto se pasa a Luminosidad respecto a la del Sol --> LS = 3.83MW

exo_zone["LUM"] = (4*np.pi*(exo_zone["RSTAR"]**2)*sigma*(exo_zone["TEFF"]**4))/sun_lum 

Límites interior (ri) y exterior (ro) de la zona de habitabilidad:

$$ ri = [r_{is} − a_i(T_{eff} − T_s) − bi(T_{eff} − T_s)^2] \sqrt{L}$$
$$ ro = [r_{os} − a_o(T_{eff} − T_s) − bo(T_{eff} − T_s)^2] \sqrt{L}$$

In [ ]:
#Límites de la zona de habitabilidad y asigne a estos las columnas ri y ro.
exo_zone["RI"] = (ris - ai*(exo_zone["TEFF"] - Ts) - bi*(exo_zone["TEFF"] - Ts)**2)*exo_zone["LUM"].apply(np.sqrt)
exo_zone["RO"] =(ros - ao*(exo_zone["TEFF"] - Ts) - bo*(exo_zone["TEFF"] - Ts)**2)*exo_zone["LUM"].apply(np.sqrt)

In [ ]:
#Reinicio índice y observo el nuevo dataframe
exo_zone.reset_index(inplace=True)
exo_zone.head()

In [ ]:
exo_zone.shape

In [ ]:
#Filtro que seleccione los planetas con alta probabilidad de ser rocosos
#Que su densidad sea del orden o mayor que 5 g cm^−3
exo_zone[exo_zone["DENSITY"] > 5]

In [ ]:
#Filtro que seleccione los planetas dentro de la zona de habitabilidad.

exo_habit = exo_zone[((exo_zone["A"] > exo_zone["RI"]) & (exo_zone["A"] < exo_zone["RO"]) ) == True]
exo_habit.head()

In [ ]:
exo_habit.shape  #hay 294 en la zona de habitabilidad

Cada columna coresponde a:

- NAME
- TEFF: temperatura efectiva de la estrella.
- MASS: Masa del planeta.
- A: semieje mayor.
- DENSITY: densidad.
- R: radio del planeta.
- STAR: Nombre de la estrella.
- MSTAR: Masa de la estrella.
- RSTAR: Radio de la estrella.
- BINARY: indica si el sistema es binario o no.
- MASSE: Masa de los planetas en masas de la Tierra. 
- RE: Radio de los planetas en radios terrestres. 
- LUM: Luminosidad de la estrella respecto a la del Sol. 
- RI: Límite interior de la zona de habitabilidad 
- RO: Límite exterior de la zona de habitabilidad 

In [ ]:
#Para los planetas dentro de la zona de habitabilidad:
#Masa de las estrellas vs la distancia del planeta a su estrella
exo_habit.plot(kind='scatter',x='MSTAR',y='A',color='blue', fontsize =16, figsize=(9,7))

#Entre mas masiva, mas lejos se encuentra el planeta de la estrella

In [ ]:
#Distribucion orbital de los planetas (histograma)

exo_habit.hist(column='A', figsize=(9,7), bins=15, color="violet", xlabelsize =16, ylabelsize=16)


In [ ]:
#planeta con densidad conocida en la zona de habitabilidad
exo_habit['DENSITY'].value_counts()

In [ ]:
exo_habit['DENSITY'] = exo_habit['DENSITY']/5.51

In [ ]:
#Densidad Vs masa de las estrellas para todos los planetas 
#(dado que solo hay un planeta con densidad conocida, en la zona de habitabilidad)

exo_zone.plot(kind='scatter',x='DENSITY',y='MSTAR',color='green', fontsize =16, figsize=(9,7))

#hay muchos más planetas de baja densidad, probablemente gaseosos, para todo tipo de masas de estrellas

In [ ]:
#Distancia a la estrella Vs Temperatura efectiva.

exo_habit.plot(kind='scatter',x='A',y='TEFF', fontsize =16, figsize=(9,7))